In [1]:
import numpy as np
import galois

# R1CS to QAP

GF = galois.GF(67)

A = np.array([
    [0, 1, 0, 0, 0, 0],  # 约束 1
    [0, 0, 1, 0, 0, 0],  # 约束 2
    [0, 1, 0, 1, 0, 0],  # 约束 3
    [5, 0, 0, 0, 1, 0],  # 约束 4
])


B = np.array([
    [0, 1, 0, 0, 0, 0],  # 约束 1
    [0, 1, 0, 0, 0, 0],  # 约束 2
    [1, 0, 0, 0, 0, 0],  # 约束 3
    [1, 0, 0, 0, 0, 0],  # 约束 4
])

C = np.array([
    [0, 0, 1, 0, 0, 0],  # 约束 1
    [0, 0, 0, 1, 0, 0],  # 约束 2
    [0, 0, 0, 0, 1, 0],  # 约束 3
    [0, 0, 0, 0, 0, 1],  # 约束 4
])

A_galois = GF(A)
B_galois = GF(B)
C_galois = GF(C)

x = GF(3)
v1 = x * x
v2 = v1 * x
v3 = v2 + x
y = v3 + GF(5)    

witness = GF(np.array([1, x, v1, v2, v3, y]))

print(witness)

assert all(np.equal(np.matmul(A_galois, witness) * np.matmul(B_galois, witness), np.matmul(C_galois, witness))), "not equal"


OMP: Info #270: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


[ 1  3  9 27 30 35]


In [2]:
## low-degree extension for A, B, C
def interpolate_column(col):
    xs = GF(np.array([1,2,3,4]))
    return galois.lagrange_poly(xs, col)

# axis 0 is the columns. apply_along_axis is the same as doing a for loop over the columns and collecting the results in an array
A_polys = np.apply_along_axis(interpolate_column, 0, A_galois)
B_polys = np.apply_along_axis(interpolate_column, 0, B_galois)
C_polys = np.apply_along_axis(interpolate_column, 0, C_galois)

# matrix A becomes 6 polynomials
print("poly A: ", A_polys)
print("poly B: ", B_polys)
print("poly C: ", C_polys)

# poly A:  
#  [Poly(12x^3 + 62x^2 + 65x + 62, GF(67))
#  Poly(44x^3 + 5x^2 + 11x + 8, GF(67))
#  Poly(34x^3 + 63x^2 + 43x + 61, GF(67))
#  Poly(33x^3 + 37x^2 + 60x + 4, GF(67))
#  Poly(56x^3 + 66x^2 + 13x + 66, GF(67)) 
#  Poly(0, GF(67))]
# 

poly A:  [Poly(12x^3 + 62x^2 + 65x + 62, GF(67))
 Poly(44x^3 + 5x^2 + 11x + 8, GF(67))
 Poly(34x^3 + 63x^2 + 43x + 61, GF(67))
 Poly(33x^3 + 37x^2 + 60x + 4, GF(67))
 Poly(56x^3 + 66x^2 + 13x + 66, GF(67)) Poly(0, GF(67))]
poly B:  [Poly(22x^3 + 36x^2 + 6x + 3, GF(67))
 Poly(45x^3 + 31x^2 + 61x + 65, GF(67)) Poly(0, GF(67)) Poly(0, GF(67))
 Poly(0, GF(67)) Poly(0, GF(67))]
poly C:  [Poly(0, GF(67)) Poly(0, GF(67)) Poly(11x^3 + 35x^2 + 18x + 4, GF(67))
 Poly(34x^3 + 63x^2 + 43x + 61, GF(67))
 Poly(33x^3 + 37x^2 + 60x + 4, GF(67))
 Poly(56x^3 + 66x^2 + 13x + 66, GF(67))]


In [3]:
## get A B C term
from functools import reduce

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

A_term = inner_product_polynomials_with_witness(A_polys, witness)
B_term = inner_product_polynomials_with_witness(B_polys, witness)
C_term = inner_product_polynomials_with_witness(C_polys, witness)

print(A_term * B_term - C_term)

4x^6 + 18x^5 + 3x^4 + 13x^3 + 38x^2 + 12x + 46


In [4]:
## Target polynomial T = (x-1)(x-2)(x-3)(x-4)
T = galois.Poly([1, 67-1], field = GF) * galois.Poly([1, 67-2], field = GF) * galois.Poly([1, 67-3], field = GF) * galois.Poly([1, 67-4], field = GF)

## Quotient polynomial Q = (A * B - C) / T
Q = (A_term * B_term - C_term) // T

print(T * Q)

poly_big = A_term * B_term - C_term
assert poly_big // T * T == poly_big

4x^6 + 18x^5 + 3x^4 + 13x^3 + 38x^2 + 12x + 46


In [5]:
## wrong witness

witness_wrong = GF(np.array([1, 3, 9, 27, 30, 36]))

A_term_wrong = inner_product_polynomials_with_witness(A_polys, witness_wrong)
B_term_wrong = inner_product_polynomials_with_witness(B_polys, witness_wrong)
C_term_wrong = inner_product_polynomials_with_witness(C_polys, witness_wrong)

print(A_term_wrong * B_term_wrong - C_term_wrong)

poly_big_wrong = A_term_wrong * B_term_wrong - C_term_wrong
assert poly_big_wrong // T * T == poly_big_wrong
## Error: A*B - C can not divide T

4x^6 + 18x^5 + 3x^4 + 24x^3 + 39x^2 + 66x + 47


AssertionError: 

In [9]:
## Linear PCP
# random number generation, we choose 6 for education purpose
r_random = GF(6)
# check A, B, C
A_random = A_term(r_random)
B_random = B_term(r_random)
C_random = C_term(r_random)
Q_random = Q(r_random)
T_random = T(r_random)

print("A_random: ", A_random)
print("B_random: ", B_random)
print("C_random: ", C_random)
print("Q_random: ", Q_random)
print("T_random: ", T_random)

assert A_random * B_random - C_random == Q_random * T_random
# pass

A_random:  7
B_random:  23
C_random:  52
Q_random:  64
T_random:  53


In [8]:
# Soundness: wrong witness
## Linear PCP
# random number generation, we choose 6 for education purpose
r_random = GF(6)
# check A, B, C
A_wrong_random = A_term_wrong(r_random)
B_wrong_random = B_term_wrong(r_random)
C_wrong_random = C_term_wrong(r_random)
Q_random = Q(r_random)
T_random = T(r_random)

print("A_wrong_random: ", A_wrong_random)
print("B_wrong_random: ", B_wrong_random)
print("C_wrong_random: ", C_wrong_random)
print("Q_random: ", Q_random)
print("T_random: ", T_random)

assert A_wrong_random * B_wrong_random - C_wrong_random == Q_random * T_random
# Error: wrong witness, A*B - C != Q * T

AssertionError: 

In [10]:
A_random * B_random - C_random

GF(42, order=67)